### Inserting the data into a mySQL Database
---

1. Connect to the database
2. Create table <br>
3. Create unique index (avoid duplicates)
4. Clean data 
5. Insert data

### 1. Connect to the database
---

All parameters are stored in config.json

In [ ]:
def connect(self,
            host=config['databases']["mpobDB"]["host"],
            user=config['databases']["mpobDB"]['user'],
            password=config['databases']["mpobDB"]['password'],
            db=config['databases']["mpobDB"]['db']):
    """
    Creates a cursor object to interact with the DB instance
    """
    self.conn = MySQLdb.connect(host=host,
                                user=user,
                                passwd=password,
                                db=db)
    self.cursor = self.conn.cursor()
    print(self.conn)

### 2. Create table
---

Here, we create the table based on the schema specified in config.json

We use a for loop to generate a string with the mySQL syntax of 

`col col_type, col2 col2_type`  

Finally, we execute the query:
```
CREATE TABLE IF NOT EXISTS
TABLE_NAME
(col col_type, col2 col2_type ... )
```

In [ ]:
def create(self):
    schema = ''

    for col, col_type in self.schema.items():
        schema += col + ' ' + col_type + ', '

    schema = schema[:-2]

    # TODO: How to check if table already exists?
    # print("Table already exists.")

    query = f"""CREATE TABLE IF NOT EXISTS
                {self.table}
                ({schema})"""

    self.cursor.execute(query)
    print(f"""Successfully created table: {self.table} with schema {schema}""")

### 3. Create unique index (to avoid duplicates)
---

We need to run the following:

```ALTER TABLE pk_recovery ADD UNIQUE INDEX month_rate(`month`, `sector`);```

This will add a unique paired index of month and sector, ensuring that cron scripts will not insert duplicated data for the same month and sector twice. 

### 4. Clean data
---

We create a function that appends the report number and the time that the script was run and rename the columns according to the schema specified in config.json

In [1]:
def clean_df(report):
    """
    Add report number and time updated columns to the df
    Time updated is based on the system time that the script runs
    """
    dfs = get_report_data(report)
    final_df = pd.concat(dfs)
    final_df['report'] = report
    final_df['updated'] = time.strftime('%Y-%m-%d %H:%M:%S')
    return final_df.drop_duplicates()\
                   .rename(
                       columns={
                           'Delivery': 'month',
                           'Sector': 'sector',
                           'Utilization (%)': 'pk_recovery_rate'})

### 5. Insert data
---

Lastly, we generate the strings that follow mySQL syntax:

```
INSERT IGNORE INTO table (col1, col2, col3)
VALUES (val1, val2, val3
ON DUPLICATE KEY UPDATE col1 = val1, col2 = val2, col3 = val3
```
The last line is controlled by a boolean flag. 

If update="Y", existing values are overwritten. (Good for data analysis - only the most updated values are shown)

If update="N", existing values are kept, and a new value is inserted (even if the index is duplicated). (Good for tracking when MPOB changes their average prices).

This ensures that for the unique paired keys (month, sector), only the latest values will appear.

See this for differences between `REPLACE INTO`, `INSERT IGNORE INTO` AND `INSERT INTO`:
https://stackoverflow.com/questions/812437/mysql-ignore-insert-error-duplicate-entry


Check that the index exists: `SHOW INDEX FROM table_name;`


In [ ]:
def insert(self, row, update):
        """
        Function to insert data by row into database
        If update flag = "Y", function will update 
        existing values on duplicate key
        """
        
        # Generate strings to insert into db
        schema = ''
        value_string = ''
        update_string = ''
        
        for col, col_type in self.schema.items():
            
            schema += col + ', '

            # If the column type is VARCHAR or DATE, use a str repr
            if self.schema[col] in ['VARCHAR(255)', 'DATE', 'DATETIME']:
                values = repr(row[col])
                value_string += values + ', '

            # If not, just use str
            else:
                values = str(row[col])
                value_string += values + ', '

            # Update existing columns with new values (if values changed)
            update_string += col + ' = ' + values + ', '

        schema = schema[:-2]
        value_string = value_string[:-2]
        update_string = update_string[:-2]

        query = f"""
                INSERT IGNORE INTO {self.table} ({schema})
                VALUES ({value_string})
                """
                
        if update == 'Y':
            query += f"""ON DUPLICATE KEY UPDATE {update_string}"""
        else:
            pass
        
        self.cursor.execute(query)
        self.conn.commit()

In [ ]:
# Connect to the database and create table if not exists
db = mysql.DB('pk_recovery')
db.connect()
db.create()

# Loop through the final df and insert each row into the database
for idx, row in final_df.iterrows():
    db.insert(row)

#### Designing your database tables

Specify your requirements first.

If you need the latest updated figures without tracking when the last figure changed, use `ON DUPLICATE KEY UPDATE`. The existing figures in the database will be replaced.

If you need to track when the last figure changed, just set index and use `INSERT INTO` only. Drop the last row `ON DUPLICATE KEY UPDATE`.  The existing figures in the database will not be replaced.

Test this: set a unique index on (date, price) from MPOB Peninsular daily prices and track to see if cron inserts a different (date, price) pair using the `updated` column